In [ ]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_id = "/workspace/pretuned_llama"
model = LlamaForCausalLM.from_pretrained(model_id).eval().to(torch.device('cuda'))
tokenizer = AutoTokenizer.from_pretrained(model_id)

# path to the 2k steps checkpoint with or without reward dropout
model = PeftModel.from_pretrained(model, "/workspace/with_dropout_2000")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# from rl training part from the rewards.py
from rewards import Rewards

r = Rewards()

def do_rewrite(inp):
    message_log = [
        {"role": "user", "content": inp},
    ]
    text_messages = tokenizer.apply_chat_template(message_log, add_generation_prompt=True, tokenize=False)
    text_messages = "<|begin_of_text|>"+text_messages.split("<|eot_id|>",1)[1]
    inputs = tokenizer(text_messages,return_tensors = "pt")
    generated = model.generate(**inputs.to(torch.device('cuda')), max_new_tokens = 250, do_sample = True, temperature=0.7)
    clear_output = tokenizer.decode(generated[0]).rsplit("Rewritten:",1)[-1].rsplit("<|eot_id|>",1)[0].strip()
    r([message_log[-1]['content']], ["Rewritten:\n"+clear_output])
    r([message_log[-1]['content']], ["Rewritten:\n"+message_log[-1]['content']])
    return clear_output

In [20]:
# do GPT rewrite
from datasets import load_dataset
import random

random.seed(12)

dataset_1 = [x['response'] for x in load_dataset("RafaM97/marketing_social_media")['train']]
random.shuffle(dataset_1)
dataset_2 = [x['marketing_email'] for x in load_dataset("Isotonic/marketing_email_samples")['train']]
random.shuffle(dataset_2)
dataset_1 = dataset_1[:10]
dataset_2 = dataset_2[:10]
dataset = dataset_1+dataset_2

In [ ]:
from openai import OpenAI

def generate_openai(message, remove_template = True, style_references = None): 
    api_key = "..."
    client = OpenAI(api_key = api_key)
    temperature = 0.7
    if style_references:
        style_references = "Rewrite to make the content following the user's style. Some examples of the user's messages:\n"+"\n".join([str(i+1)+". "+x for i,x in enumerate(style_references)])
    if remove_template:
        message_log = [
            {"role": "system", "content": "You are a helpful assistant. Rewrite the message to exclude any template marks if there are any - you can insert some values there. Start you answer with the word \"Rewritten: \""},
            {"role": "user", "content": message}
        ]
    else:
        message_log = [
            {"role": "system", "content": "You are a helpful assistant. Rewrite the message. Start you answer with the word \"Rewritten: \""},
            {"role": "user", "content": message}
        ]
    if style_references:
        message_log[0]['content'] = "You are a helpful assistant.\n" + style_references + "\n" + message_log[0]['content'].split("You are a helpful assistant.")[-1].strip()
    response = client.chat.completions.create(
                model='gpt-4o-mini', 
                messages=message_log,   
                max_tokens=512,
                temperature=temperature,      
            )
    return response.choices[0].message.content.rsplit("Rewritten:", 1)[-1].strip()

style_examples = [
            "Good evening Sir or Madam, I would like to introduce myself.",
            "Lori's gonna rock being boss at HiTOPS; she'll seriously push things forward.", # informal# "Hey dude, what's up? Such a funny story I have in my mind to share",
            "Wow :-), I'll advocate for Blanco's dedication to ethical business, and CRT membership =D!" # Text emojies #"Telling the truth - I really hate it, if you do it one more time you will experience consequences!"
        ]

# augment data and generate GPT responses with few-shots-based style cloning
synthetic_inputs = [generate_openai(x) for x in dataset]
restyled = [generate_openai(x, style_references = style_examples) for x in dataset+synthetic_inputs]


In [ ]:
from tqdm import tqdm

model_rewritten = []
with tqdm(total = len(dataset+synthetic_inputs)) as pbar:
    for x in dataset+synthetic_inputs:
        model_rewritten.append(do_rewrite(x))
        pbar.update(1)

In [94]:
r = Rewards()

In [95]:
results = []
with tqdm(total=len(dataset+synthetic_inputs)) as pbar:
    for my, gpt, inp in zip(model_rewritten, restyled, dataset+synthetic_inputs):
        results.append({
            "my_scores": r([inp], ["Rewritten:\n"+my], return_dict=True),
            "gpt_scores": r([inp], ["Rewritten:\n"+gpt], return_dict=True),
            "input_scores": r([inp], ["Rewritten:\n"+inp], return_dict=True),
            "input_text": inp,
            "rewritten_text_gpt": gpt, 
            "rewritten_text_my": my, 
        })
        pbar.update(1)

  0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_14790/1193313071.py:169: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  entropy_outer = torch.std(p_outer, dim = 1)#-1*torch.sum(p_outer*torch.log(p_outer), dim = 1)
100%|██████████| 40/40 [00:15<00:00,  2.55it/s]


In [97]:
import json

with open("with_dropout_2000.jsonl","w") as file:
    for obj in results:
        file.write(json.dumps(obj)+"\n")